# ICE Building Workflow - Knowledge Graph Construction

**Purpose**: Comprehensive data ingestion and knowledge graph building for investment intelligence
**Architecture**: ICE Simplified (2,508 lines) with LightRAG integration
**Input**: Financial data from multiple sources → **Output**: Searchable knowledge graph

## Workflow Overview

1. **Environment Setup** - Initialize ICE system and configure data sources
2. **Workflow Mode Selection** - Choose between initial build or incremental update
3. **Data Ingestion** - Fetch financial data from APIs and process documents
4. **Knowledge Graph Building** - Extract entities, relationships, and build LightRAG graph
5. **Storage & Validation** - Verify graph construction and monitor storage
6. **Metrics & Monitoring** - Track processing metrics and system health

---

## 1. Environment Setup & System Initialization

In [1]:
# Setup and imports
import sys
import os
from pathlib import Path
from datetime import datetime, timedelta
import json

# Add project root to path
project_root = Path.cwd()
sys.path.insert(0, str(project_root))

# Configure environment
os.environ.setdefault('ICE_WORKING_DIR', './src/ice_lightrag/storage')

print(f"🚀 ICE Building Workflow")
print(f"📅 {datetime.now().strftime('%Y-%m-%d %H:%M')}")
print(f"📁 Working Directory: {project_root}")

🚀 ICE Building Workflow
📅 2025-10-13 22:09
📁 Working Directory: /Users/royyeo/Library/CloudStorage/OneDrive-NationalUniversityofSingapore/Capstone Project


In [2]:
# Initialize ICE system
from updated_architectures.implementation.ice_simplified import create_ice_system

try:
    ice = create_ice_system()
    system_ready = ice.is_ready()
    print(f"✅ ICE System Initialized")
    print(f"🧠 LightRAG Status: {'Ready' if system_ready else 'Initializing'}")
    print(f"📊 Architecture: ICE Simplified (2,508 lines)")
    print(f"🔗 Components: Core + Ingester + QueryEngine")
except Exception as e:
    print(f"❌ Initialization Error: {e}")
    raise  # Let errors surface for proper debugging

INFO:ice_data_ingestion.ice_integration:ICE LightRAG system initialized successfully
INFO:ice_data_ingestion.ice_integration:ICE LightRAG system initialized successfully
INFO:updated_architectures.implementation.ice_simplified:ICE Core initializing with ICESystemManager orchestration
INFO:src.ice_core.ice_system_manager:ICE System Manager initialized with working_dir: ice_lightrag/storage
INFO:updated_architectures.implementation.ice_simplified:✅ ICESystemManager initialized successfully
INFO:updated_architectures.implementation.ice_simplified:Data Ingester initialized with 4 API services
INFO:updated_architectures.implementation.ice_simplified:Query Engine initialized
INFO:updated_architectures.implementation.ice_simplified:✅ ICE Simplified system initialized successfully
INFO:src.ice_core.ice_system_manager:LightRAG wrapper created successfully (lazy initialization mode)
INFO:src.ice_core.ice_system_manager:Exa MCP connector initialized successfully
INFO:src.ice_core.ice_graph_builde

✅ LightRAG successfully imported!
✅ ICE System Initialized
🧠 LightRAG Status: Ready
📊 Architecture: ICE Simplified (2,508 lines)
🔗 Components: Core + Ingester + QueryEngine


In [3]:
# Verify storage architecture and components
print(f"📦 LightRAG Storage Architecture Verification")
print(f"━" * 40)

if not (ice and ice.core.is_ready()):
    raise RuntimeError("ICE system not ready - cannot verify storage")

# Get storage statistics using new method
storage_stats = ice.core.get_storage_stats()

print(f"LightRAG Storage Components:")
for component_name, component_info in storage_stats['components'].items():
    status = "✅ Initialized" if component_info['exists'] else "⚠️ Not created yet"
    size_mb = component_info['size_bytes'] / (1024 * 1024) if component_info['size_bytes'] > 0 else 0
    print(f"  {component_name}: {status}")
    print(f"    Purpose: {component_info['description']}")
    print(f"    File: {component_info['file']}")
    if size_mb > 0:
        print(f"    Size: {size_mb:.2f} MB")

print(f"\n📁 Working Directory: {storage_stats['working_dir']}")
print(f"🗄️ Storage Backend: File-based (development mode)")
print(f"💾 Total Storage: {storage_stats['total_storage_bytes'] / (1024 * 1024):.2f} MB")

📦 LightRAG Storage Architecture Verification
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
LightRAG Storage Components:
  chunks_vdb: ⚠️ Not created yet
    Purpose: Vector database for document chunks
    File: vdb_chunks.json
  entities_vdb: ⚠️ Not created yet
    Purpose: Vector database for extracted entities
    File: vdb_entities.json
  relationships_vdb: ⚠️ Not created yet
    Purpose: Vector database for entity relationships
    File: vdb_relationships.json
  graph: ⚠️ Not created yet
    Purpose: NetworkX graph structure
    File: graph_chunk_entity_relation.graphml

📁 Working Directory: ice_lightrag/storage
🗄️ Storage Backend: File-based (development mode)
💾 Total Storage: 0.00 MB


In [4]:
# Data sources configuration status
if not (ice and hasattr(ice, 'ingester')):
    raise RuntimeError("Data ingester not initialized")

available_services = ice.ingester.available_services
print(f"\n📡 Data Sources Available: {len(available_services)}")
for service in available_services:
    print(f"  ✅ {service}")

if not available_services:
    print(f"  ⚠️ No APIs configured - will use sample data")
    print(f"  💡 Set NEWSAPI_ORG_API_KEY for real news")
    print(f"  💡 Set ALPHA_VANTAGE_API_KEY for financial data")

# Validate OpenAI for LightRAG
openai_configured = bool(os.getenv('OPENAI_API_KEY'))
print(f"\n🔑 OpenAI API: {'✅ Configured' if openai_configured else '❌ Required for full functionality'}")


📡 Data Sources Available: 4
  ✅ alpha_vantage
  ✅ newsapi
  ✅ polygon
  ✅ finnhub

🔑 OpenAI API: ✅ Configured


## 2. Workflow Mode Selection & Configuration

### 🔧 Model Provider Configuration

ICE supports **OpenAI** (paid) or **Ollama** (free local) for LLM and embeddings:

#### Option 1: OpenAI (Default - No setup required)
```bash
export OPENAI_API_KEY="sk-..."
```
- **Cost**: ~$5/month for typical usage
- **Quality**: Highest accuracy for entity extraction and reasoning
- **Setup**: Just set API key

#### Option 2: Ollama (Free Local - Requires setup)
```bash
# Set provider
export LLM_PROVIDER="ollama"

# One-time setup:
ollama serve                      # Start Ollama service
ollama pull qwen3:30b-32k        # Pull LLM model (32k context required)
ollama pull nomic-embed-text      # Pull embedding model
```
- **Cost**: $0/month (completely free)
- **Quality**: Good for most investment analysis tasks
- **Setup**: Requires local Ollama installation and model download

#### Option 3: Hybrid (Recommended for cost-conscious users)
```bash
export LLM_PROVIDER="ollama"           # Use Ollama for LLM
export EMBEDDING_PROVIDER="openai"     # Use OpenAI for embeddings
export OPENAI_API_KEY="sk-..."
```
- **Cost**: ~$2/month (embeddings only)
- **Quality**: Balanced - free LLM with high-quality embeddings

**Current configuration will be logged when you run the next cell.**

In [5]:
# ### Provider Switching - Uncomment ONE option below, then restart kernel

#############################################################################
#                              Model Selection                              #
#############################################################################

### Option 1: OpenAI ($5/mo, highest quality)
import os; os.environ['LLM_PROVIDER'] = 'openai'
print("✅ Switched to OpenAI")

# ###Option 2: Hybrid ($2/mo, 60% savings, recommended)
# import os; os.environ['LLM_PROVIDER'] = 'ollama'; os.environ['EMBEDDING_PROVIDER'] = 'openai'
# print("✅ Switched to Hybrid")

# ### Option 3: Full Ollama ($0/mo, faster model)
# import os; os.environ['LLM_PROVIDER'] = 'ollama'; os.environ['EMBEDDING_PROVIDER'] = 'ollama'; 
# # os.environ['LLM_MODEL'] = 'llama3.1:8b'
# os.environ['LLM_MODEL'] = 'qwen3:8b'
# # os.environ['LLM_MODEL'] = 'qwen3:14b'
# # os.environ['LLM_MODEL'] = 'qwen3:30b-32k'
# print("✅ Switched to Full Ollama Model.")

✅ Switched to OpenAI


### 🗑️ Graph Management (Optional)

**When to clear the graph:**
- ✅ Switching to Full Ollama (1536-dim → 768-dim embeddings)
- ✅ Graph corrupted or very old (>30 days without updates)
- ✅ Testing fresh graph builds from scratch

**When NOT to clear:**
- ❌ Just switching LLM provider (OpenAI ↔ Hybrid use same embeddings)
- ❌ Adding new documents (incremental updates work fine)
- ❌ Changing query modes (local, hybrid, etc.)

**How to clear:**
Run the code cell below (uncomment lines to activate)

In [18]:
##########################################################
#                    Check graph info                    #
##########################################################
from updated_architectures.implementation.ice_simplified import create_ice_system
ice = create_ice_system()


##########################################################
from pathlib import Path
import shutil

def check_storage(storage_path):
    """Check and display storage file inventory"""
    files = ['vdb_entities.json', 'vdb_relationships.json', 'vdb_chunks.json', 'graph_chunk_entity_relation.graphml']
    total_size = 0
    for fname in files:
        fpath = storage_path / fname
        if fpath.exists():
            size_mb = fpath.stat().st_size / (1024 * 1024)
            print(f"  ✅ {fname}: {size_mb:.2f} MB")
            total_size += size_mb
        else:
            print(f"  ⚠️  {fname}: not found")
    print(f"  💾 Total: {total_size:.2f} MB")

# Use actual config path instead of hardcoded path to avoid path mismatches
storage_path = Path(ice.config.working_dir)

check_storage(storage_path)

#####################################################################
ice.core.get_graph_stats()

##########################################################
#              Graph Health Metrics (P0)                 #
##########################################################

def check_graph_health(storage_path):
    """Check critical graph health metrics (P0 only)"""
    import json
    from pathlib import Path
    
    TICKERS = {'NVDA', 'TSMC', 'AMD', 'ASML'}  # Known portfolio tickers
    
    result = {
        'tickers_covered': set(),
        'total_entities': 0,
        'total_relationships': 0,
        'buy_signals': 0,
        'sell_signals': 0,
        'price_targets': 0
    }
    
    # Parse entities
    entities_file = Path(storage_path) / 'vdb_entities.json'
    if entities_file.exists():
        data = json.loads(entities_file.read_text())
        result['total_entities'] = len(data.get('data', []))
        
        for entity in data.get('data', []):
            text = f"{entity.get('entity_name', '')} {entity.get('content', '')}".upper()
            
            # Detect tickers
            for ticker in TICKERS:
                if ticker in text:
                    result['tickers_covered'].add(ticker)
            
            # Detect signals
            if 'BUY' in text:
                result['buy_signals'] += 1
            if 'SELL' in text:
                result['sell_signals'] += 1
            if 'PRICE TARGET' in text or 'PRICE_TARGET' in text:
                result['price_targets'] += 1
    
    # Parse relationships
    rels_file = Path(storage_path) / 'vdb_relationships.json'
    if rels_file.exists():
        data = json.loads(rels_file.read_text())
        result['total_relationships'] = len(data.get('data', []))
    
    result['tickers_covered'] = sorted(list(result['tickers_covered']))
    return result

# Run health check
health = check_graph_health(storage_path)

# Display results
print("\n🧬 Graph Health Metrics:")
print(f"  📊 Content Coverage:")
print(f"    Tickers: {', '.join(health['tickers_covered']) if health['tickers_covered'] else 'None'} ({len(health['tickers_covered'])}/4 portfolio holdings)")

print(f"\n  🕸️ Graph Structure:")
print(f"    Total entities: {health['total_entities']:,}")
print(f"    Total relationships: {health['total_relationships']:,}")
if health['total_entities'] > 0:
    avg_conn = health['total_relationships'] / health['total_entities']
    print(f"    Avg connections: {avg_conn:.2f}")

print(f"\n  💼 Investment Signals:")
print(f"    BUY signals: {health['buy_signals']}")
print(f"    SELL signals: {health['sell_signals']}")
print(f"    Price targets: {health['price_targets']}")

INFO:updated_architectures.implementation.ice_simplified:ICE Core initializing with ICESystemManager orchestration
INFO:src.ice_core.ice_system_manager:ICE System Manager initialized with working_dir: ice_lightrag/storage
INFO:updated_architectures.implementation.ice_simplified:✅ ICESystemManager initialized successfully
INFO:updated_architectures.implementation.ice_simplified:Data Ingester initialized with 4 API services
INFO:updated_architectures.implementation.ice_simplified:Query Engine initialized
INFO:updated_architectures.implementation.ice_simplified:✅ ICE Simplified system initialized successfully
INFO:src.ice_core.ice_system_manager:LightRAG wrapper created successfully (lazy initialization mode)
INFO:src.ice_core.ice_system_manager:Exa MCP connector initialized successfully
INFO:src.ice_core.ice_graph_builder:ICE Graph Builder initialized
INFO:src.ice_core.ice_graph_builder:LightRAG instance updated in Graph Builder
INFO:src.ice_core.ice_system_manager:Graph Builder initiali

  ✅ vdb_entities.json: 4.40 MB
  ✅ vdb_relationships.json: 4.01 MB
  ✅ vdb_chunks.json: 0.72 MB
  ✅ graph_chunk_entity_relation.graphml: 0.32 MB
  💾 Total: 9.46 MB

🧬 Graph Health Metrics:
  📊 Content Coverage:
    Tickers: NVDA (1/4 portfolio holdings)

  🕸️ Graph Structure:
    Total entities: 368
    Total relationships: 337
    Avg connections: 0.92

  💼 Investment Signals:
    BUY signals: 9
    SELL signals: 1
    Price targets: 0


## 🧪 Test: Hybrid Entity Categorization with Qwen2.5-3B

**Purpose**: Compare categorization accuracy between keyword-only and hybrid (keyword+LLM) approaches

**What this tests**:
- Baseline: Fast keyword pattern matching (~1ms per entity)
- Enhanced: Confidence scoring to identify ambiguous cases
- Hybrid: LLM fallback for low-confidence entities (~40ms per entity)

**Prerequisites**:
- ✅ Ollama installed with qwen2.5:3b model (optional - degrades gracefully)
- ✅ LightRAG graph built (previous cells completed)

**Expected runtime**: ~0.5 seconds for 12 sample entities (hybrid mode)

**Configuration**: `src/ice_lightrag/graph_categorization.py` - Change `CATEGORIZATION_MODE` to enable hybrid by default

In [19]:
# Purpose: Test hybrid entity categorization with configurable sampling and model selection
# Location: ice_building_workflow.ipynb Cell 12 (after Cell 10)
# Dependencies: graph_categorization.py, entity_categories.py, LightRAG storage

import json
import time
import sys
import random
import requests
from pathlib import Path
from collections import Counter

# Force reload of graph_categorization module to pick up new functions
import importlib
if 'src.ice_lightrag.graph_categorization' in sys.modules:
    importlib.reload(sys.modules['src.ice_lightrag.graph_categorization'])

# ===== CONFIGURATION: User-editable settings =====
RANDOM_SEED = 42  # Set to None for different entities each run, or keep 42 for reproducible testing
OLLAMA_MODEL_OVERRIDE = 'qwen2.5:3b'  # Change to use different model (e.g., 'llama3.1:8b', 'qwen3:8b')

# ===== SETUP: Imports with error handling =====
print("=" * 70)
print("🧪 Entity Categorization Test (4 Modes)")
print("=" * 70)

# Ensure src is in path for notebook context
project_root = Path.cwd()
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

try:
    from src.ice_lightrag.graph_categorization import (
        categorize_entity,
        categorize_entity_with_confidence,
        categorize_entity_hybrid,
        categorize_entity_llm_only  # NEW: Pure LLM mode
    )
    from src.ice_lightrag.entity_categories import CATEGORY_DISPLAY_ORDER
    print("✅ Categorization functions imported successfully")
except ImportError as e:
    print(f"❌ Import error: {e}")
    print("   → Ensure previous cells completed successfully")
    print("   → Check that src/ice_lightrag/graph_categorization.py exists\n")
    raise

# Patch module constant for Ollama model selection
import src.ice_lightrag.graph_categorization as graph_cat_module
graph_cat_module.OLLAMA_MODEL = OLLAMA_MODEL_OVERRIDE
print(f"🔧 Ollama model configured: {OLLAMA_MODEL_OVERRIDE}\n")

# ===== HEALTH CHECK: Ollama service availability =====
def check_ollama_service():
    """Check if Ollama service is running and configured model is available"""
    try:
        response = requests.get("http://localhost:11434/api/tags", timeout=2)
        if response.status_code == 200:
            models = response.json().get('models', [])
            # Use exact match to avoid accepting wrong model versions
            model_available = any(m.get('name', '') == OLLAMA_MODEL_OVERRIDE for m in models)
            return True, model_available
        return False, False
    except requests.RequestException:
        return False, False
    except (KeyError, json.JSONDecodeError):
        return True, False

ollama_running, model_available = check_ollama_service()
if ollama_running and model_available:
    print(f"✅ Ollama service running with {OLLAMA_MODEL_OVERRIDE} model")
elif ollama_running:
    print(f"⚠️  Ollama running but {OLLAMA_MODEL_OVERRIDE} not found")
    print(f"   → Install: ollama pull {OLLAMA_MODEL_OVERRIDE}")
else:
    print("⚠️  Ollama not running - TEST 3 & 4 will be skipped")
    print("   → Start Ollama: brew services start ollama (macOS)\n")

# ===== DATA LOADING: Entities with validation =====
storage_path = Path(ice.config.working_dir) / "vdb_entities.json"

if not storage_path.exists():
    print(f"❌ Storage file not found: {storage_path}")
    print("   → Run previous cells to build the knowledge graph first\n")
    raise FileNotFoundError(f"LightRAG storage not found at {storage_path}")

with open(storage_path) as f:
    entities_data = json.load(f)

if not isinstance(entities_data, dict) or 'data' not in entities_data:
    print(f"❌ Invalid storage format (expected dict with 'data' key)")
    raise ValueError("Invalid LightRAG storage format - expected {'data': [...]}")

entities_list = entities_data.get('data', [])
if not isinstance(entities_list, list) or len(entities_list) == 0:
    print(f"❌ No entities found in storage")
    raise ValueError("No entities found in LightRAG storage")

print(f"✅ Loaded {len(entities_list)} entities from knowledge graph")

# Configurable random sampling
if RANDOM_SEED is not None:
    random.seed(RANDOM_SEED)
    sampling_mode = f"Reproducible (seed={RANDOM_SEED}, same entities each run)"
else:
    sampling_mode = "Random (different entities each run)"

test_entities = random.sample(entities_list, min(12, len(entities_list)))
print(f"   Sampling mode: {sampling_mode}")
print(f"   Testing with {len(test_entities)} entities")
print(f"   💡 To change: Set RANDOM_SEED = None (line 17) or OLLAMA_MODEL_OVERRIDE (line 18)\n")

# ===== HELPER: Compact result display =====
def display_results(results, title, show_confidence=False, show_llm=False):
    """Display categorization results in compact format"""
    print(f"\n{title}")
    print("-" * 70)

    for i, (name, category, confidence, used_llm) in enumerate(results, 1):
        display_name = name[:40] + "..." if len(name) > 40 else name

        if show_llm and used_llm:
            indicator = "🤖"
        else:
            indicator = "⚡"

        if show_confidence:
            print(f"{i:2d}. {indicator} {display_name:43s} → {category:20s} (conf: {confidence:.2f})")
        else:
            print(f"{i:2d}. {display_name:45s} → {category}")

    category_counts = Counter(cat for _, cat, _, _ in results)
    print(f"\n📊 Distribution: {dict(category_counts)}")

# ===== TEST 1: Keyword-Only Baseline =====
print("\n" + "=" * 70)
print("TEST 1: Keyword-Only Categorization (Baseline)")
print("=" * 70)

start_time = time.time()
keyword_results = []

for entity in test_entities:
    name = entity.get('entity_name', '')
    content = entity.get('content', '')
    category = categorize_entity(name, content)
    keyword_results.append((name, category, 1.0, False))

elapsed = time.time() - start_time
display_results(keyword_results, "Results (Keyword Matching):", show_confidence=False)
print(f"\n⏱️  Time: {elapsed*1000:.1f}ms ({elapsed*1000/len(test_entities):.1f}ms per entity)")

# ===== TEST 2: Confidence Scoring Analysis =====
print("\n" + "=" * 70)
print("TEST 2: Keyword + Confidence Scoring")
print("=" * 70)

start_time = time.time()
confidence_results = []

for entity in test_entities:
    name = entity.get('entity_name', '')
    content = entity.get('content', '')
    category, confidence = categorize_entity_with_confidence(name, content)
    confidence_results.append((name, category, confidence, False))

elapsed = time.time() - start_time
display_results(confidence_results, "Results (with confidence scores):", show_confidence=True)

low_confidence = [(n, c, conf) for n, c, conf, _ in confidence_results if conf < 0.70]
if low_confidence:
    print(f"\n🔍 Ambiguous entities (confidence < 0.70): {len(low_confidence)}")
    for name, cat, conf in low_confidence:
        print(f"   - {name[:50]:50s} → {cat:20s} (conf: {conf:.2f})")
else:
    print(f"\n✅ All entities have high confidence (≥0.70) - no LLM fallback needed")

print(f"\n⏱️  Time: {elapsed*1000:.1f}ms ({elapsed*1000/len(test_entities):.1f}ms per entity)")

# ===== TEST 3: Hybrid Mode (if Ollama available) =====
if ollama_running and model_available:
    print("\n" + "=" * 70)
    print("TEST 3: Hybrid Categorization (Keyword + LLM Fallback)")
    print("=" * 70)
    print("⏱️  Note: LLM calls may take 5-10 seconds total...\n")

    start_time = time.time()
    hybrid_results = []
    llm_call_count = 0

    for entity in test_entities:
        name = entity.get('entity_name', '')
        content = entity.get('content', '')

        keyword_cat, keyword_conf = categorize_entity_with_confidence(name, content)

        if keyword_conf >= 0.70:
            category, confidence = keyword_cat, keyword_conf
            used_llm = False
        else:
            category, confidence = categorize_entity_hybrid(name, content, confidence_threshold=0.70)
            used_llm = (confidence == 0.90)
            if used_llm:
                llm_call_count += 1

        hybrid_results.append((name, category, confidence, used_llm))

    elapsed = time.time() - start_time
    display_results(hybrid_results, "Results (Hybrid mode - keyword + LLM):", show_confidence=True, show_llm=True)

    print(f"\n🤖 LLM calls: {llm_call_count}/{len(test_entities)} ({100*llm_call_count/len(test_entities):.1f}%)")
    print(f"⏱️  Time: {elapsed*1000:.1f}ms ({elapsed*1000/len(test_entities):.1f}ms per entity)")

    # ===== COMPARISON SUMMARY =====
    print("\n" + "=" * 70)
    print("📊 COMPARISON SUMMARY (Keyword vs Hybrid)")
    print("=" * 70)

    changes = 0
    for i in range(len(test_entities)):
        if keyword_results[i][1] != hybrid_results[i][1]:
            changes += 1

    print(f"Entities recategorized by LLM: {changes}/{len(test_entities)} ({100*changes/len(test_entities):.1f}%)")

    if changes > 0:
        print("\nRecategorization details:")
        for i in range(len(test_entities)):
            kw_cat = keyword_results[i][1]
            hyb_cat = hybrid_results[i][1]
            if kw_cat != hyb_cat:
                name = test_entities[i].get('entity_name', '')[:50]
                print(f"   - {name:50s}: {kw_cat:20s} → {hyb_cat}")

    print(f"\n✅ Hybrid categorization complete!")

else:
    print("\n" + "=" * 70)
    print("⚠️  TEST 3 SKIPPED: Ollama not available")
    print("=" * 70)
    print("To enable hybrid mode:")
    print("   1. Install Ollama: https://ollama.com")
    print(f"   2. Pull model: ollama pull {OLLAMA_MODEL_OVERRIDE}")
    print("   3. Start service: brew services start ollama (macOS)")
    print("   4. Re-run this cell")

# ===== TEST 4: Pure LLM Mode (NEW) =====
if ollama_running and model_available:
    print("\n" + "=" * 70)
    print("TEST 4: Pure LLM Categorization (All entities)")
    print("=" * 70)
    print("⏱️  Note: This will call LLM for ALL entities (may take 30-60 seconds)...\n")

    start_time = time.time()
    llm_results = []

    for entity in test_entities:
        name = entity.get('entity_name', '')
        content = entity.get('content', '')
        category, confidence = categorize_entity_llm_only(name, content)
        llm_results.append((name, category, confidence, True))

    elapsed = time.time() - start_time
    display_results(llm_results, "Results (Pure LLM mode):", show_confidence=True, show_llm=True)

    print(f"\n🤖 LLM calls: {len(test_entities)}/{len(test_entities)} (100%)")
    print(f"⏱️  Time: {elapsed*1000:.1f}ms ({elapsed*1000/len(test_entities):.1f}ms per entity)")

    # Compare keyword vs pure LLM
    llm_changes = sum(1 for i in range(len(test_entities)) if keyword_results[i][1] != llm_results[i][1])
    
    print("\n" + "=" * 70)
    print("📊 COMPARISON SUMMARY (Keyword vs Pure LLM)")
    print("=" * 70)
    print(f"Entities recategorized by pure LLM: {llm_changes}/{len(test_entities)} ({100*llm_changes/len(test_entities):.1f}%)")
    
    if llm_changes > 0:
        print("\nRecategorization details:")
        for i in range(len(test_entities)):
            kw_cat = keyword_results[i][1]
            llm_cat = llm_results[i][1]
            if kw_cat != llm_cat:
                name = test_entities[i].get('entity_name', '')[:50]
                print(f"   - {name:50s}: {kw_cat:20s} → {llm_cat}")

    print(f"\n✅ Pure LLM categorization complete!")

else:
    print("\n" + "=" * 70)
    print("⚠️  TEST 4 SKIPPED: Ollama not available")
    print("=" * 70)
    print("To enable pure LLM mode:")
    print("   1. Install Ollama: https://ollama.com")
    print(f"   2. Pull model: ollama pull {OLLAMA_MODEL_OVERRIDE}")
    print("   3. Start service: brew services start ollama (macOS)")
    print("   4. Re-run this cell")

print("\n" + "=" * 70)
print("🎯 TESTING COMPLETE - 4 Categorization Modes Evaluated")
print("=" * 70)

🧪 Entity Categorization Test (4 Modes)
✅ Categorization functions imported successfully
🔧 Ollama model configured: qwen2.5:3b

✅ Ollama service running with qwen2.5:3b model
✅ Loaded 368 entities from knowledge graph
   Sampling mode: Reproducible (seed=42, same entities each run)
   Testing with 12 entities
   💡 To change: Set RANDOM_SEED = None (line 17) or OLLAMA_MODEL_OVERRIDE (line 18)


TEST 1: Keyword-Only Categorization (Baseline)

Results (Keyword Matching):
----------------------------------------------------------------------
 1. Exxon Mobil Corporation                       → Company
 2. September 13, 2025                            → Other
 3. iPhone                                        → Company
 4. Ritholtz Wealth Management                    → Other
 5. JNJ                                           → Company
 6. Financial Historical                          → Technology/Product
 7. Chicago                                       → Other
 8. 2025-09-18                  

In [21]:
# Purpose: Test relationship categorization with configurable sampling and model selection
# Location: ice_building_workflow.ipynb Cell 12b (after Cell 12)
# Dependencies: graph_categorization.py, relationship_categories.py, LightRAG storage

import json
import time
import sys
import random
import requests
from pathlib import Path
from collections import Counter

# Force reload of graph_categorization module to pick up new functions
import importlib
if 'src.ice_lightrag.graph_categorization' in sys.modules:
    importlib.reload(sys.modules['src.ice_lightrag.graph_categorization'])

# ===== CONFIGURATION: User-editable settings =====
RANDOM_SEED = 42  # Set to None for different relationships each run, or keep 42 for reproducible testing
OLLAMA_MODEL_OVERRIDE = 'qwen2.5:3b'  # Change to use different model (e.g., 'llama3.1:8b', 'qwen3:8b')

# ===== SETUP: Imports with error handling =====
print("=" * 70)
print("🧪 Relationship Categorization Test (4 Modes)")
print("=" * 70)

# Ensure src is in path for notebook context
project_root = Path.cwd()
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

try:
    from src.ice_lightrag.graph_categorization import (
        categorize_relationship,
        categorize_relationship_with_confidence,
        categorize_relationship_hybrid,
        categorize_relationship_llm_only
    )
    from src.ice_lightrag.relationship_categories import CATEGORY_DISPLAY_ORDER, extract_relationship_types
    print("✅ Categorization functions imported successfully")
except ImportError as e:
    print(f"❌ Import error: {e}")
    print("   → Ensure previous cells completed successfully")
    print("   → Check that src/ice_lightrag/graph_categorization.py exists\n")
    raise

# Patch module constant for Ollama model selection
import src.ice_lightrag.graph_categorization as graph_cat_module
graph_cat_module.OLLAMA_MODEL = OLLAMA_MODEL_OVERRIDE
print(f"🔧 Ollama model configured: {OLLAMA_MODEL_OVERRIDE}\n")

# ===== HEALTH CHECK: Ollama service availability =====
def check_ollama_service():
    """Check if Ollama service is running and configured model is available"""
    try:
        response = requests.get("http://localhost:11434/api/tags", timeout=2)
        if response.status_code == 200:
            models = response.json().get('models', [])
            # Use exact match to avoid accepting wrong model versions
            model_available = any(m.get('name', '') == OLLAMA_MODEL_OVERRIDE for m in models)
            return True, model_available
        return False, False
    except requests.RequestException:
        return False, False
    except (KeyError, json.JSONDecodeError):
        return True, False

ollama_running, model_available = check_ollama_service()
if ollama_running and model_available:
    print(f"✅ Ollama service running with {OLLAMA_MODEL_OVERRIDE} model")
elif ollama_running:
    print(f"⚠️  Ollama running but {OLLAMA_MODEL_OVERRIDE} not found")
    print(f"   → Install: ollama pull {OLLAMA_MODEL_OVERRIDE}")
else:
    print("⚠️  Ollama not running - TEST 3 & 4 will be skipped")
    print("   → Start Ollama: brew services start ollama (macOS)\n")

# ===== DATA LOADING: Relationships with validation =====
storage_path = Path(ice.config.working_dir) / "vdb_relationships.json"

if not storage_path.exists():
    print(f"❌ Storage file not found: {storage_path}")
    print("   → Run previous cells to build the knowledge graph first\n")
    raise FileNotFoundError(f"LightRAG storage not found at {storage_path}")

with open(storage_path) as f:
    relationships_data = json.load(f)

if not isinstance(relationships_data, dict) or 'data' not in relationships_data:
    print(f"❌ Invalid storage format (expected dict with 'data' key)")
    raise ValueError("Invalid LightRAG storage format - expected {'data': [...]}")

relationships_list = relationships_data.get('data', [])
if not isinstance(relationships_list, list) or len(relationships_list) == 0:
    print(f"❌ No relationships found in storage")
    raise ValueError("No relationships found in LightRAG storage")

print(f"✅ Loaded {len(relationships_list)} relationships from knowledge graph")

# Configurable random sampling
if RANDOM_SEED is not None:
    random.seed(RANDOM_SEED)
    sampling_mode = f"Reproducible (seed={RANDOM_SEED}, same relationships each run)"
else:
    sampling_mode = "Random (different relationships each run)"

test_relationships = random.sample(relationships_list, min(12, len(relationships_list)))
print(f"   Sampling mode: {sampling_mode}")
print(f"   Testing with {len(test_relationships)} relationships")
print(f"   💡 To change: Set RANDOM_SEED = None (line 17) or OLLAMA_MODEL_OVERRIDE (line 18)\n")

# ===== HELPER: Compact result display =====
def display_results(results, title, show_confidence=False, show_llm=False):
    """Display categorization results in compact format"""
    print(f"\n{title}")
    print("-" * 70)

    for i, (name, category, confidence, used_llm) in enumerate(results, 1):
        display_name = name[:40] + "..." if len(name) > 40 else name

        if show_llm and used_llm:
            indicator = "🤖"
        else:
            indicator = "⚡"

        if show_confidence:
            print(f"{i:2d}. {indicator} {display_name:43s} → {category:20s} (conf: {confidence:.2f})")
        else:
            print(f"{i:2d}. {display_name:45s} → {category}")

    category_counts = Counter(cat for _, cat, _, _ in results)
    print(f"\n📊 Distribution: {dict(category_counts)}")

# ===== TEST 1: Keyword-Only Baseline =====
print("\n" + "=" * 70)
print("TEST 1: Keyword-Only Categorization (Baseline)")
print("=" * 70)

start_time = time.time()
keyword_results = []

for relationship in test_relationships:
    content = relationship.get('content', '')
    rel_types = extract_relationship_types(content)
    display_name = rel_types if rel_types else content[:50]
    category = categorize_relationship(content)
    keyword_results.append((display_name, category, 1.0, False))

elapsed = time.time() - start_time
display_results(keyword_results, "Results (Keyword Matching):", show_confidence=False)
print(f"\n⏱️  Time: {elapsed*1000:.1f}ms ({elapsed*1000/len(test_relationships):.1f}ms per relationship)")

# ===== TEST 2: Confidence Scoring Analysis =====
print("\n" + "=" * 70)
print("TEST 2: Keyword + Confidence Scoring")
print("=" * 70)

start_time = time.time()
confidence_results = []

for relationship in test_relationships:
    content = relationship.get('content', '')
    rel_types = extract_relationship_types(content)
    display_name = rel_types if rel_types else content[:50]
    category, confidence = categorize_relationship_with_confidence(content)
    confidence_results.append((display_name, category, confidence, False))

elapsed = time.time() - start_time
display_results(confidence_results, "Results (with confidence scores):", show_confidence=True)

low_confidence = [(n, c, conf) for n, c, conf, _ in confidence_results if conf < 0.70]
if low_confidence:
    print(f"\n🔍 Ambiguous relationships (confidence < 0.70): {len(low_confidence)}")
    for name, cat, conf in low_confidence:
        print(f"   - {name[:50]:50s} → {cat:20s} (conf: {conf:.2f})")
else:
    print(f"\n✅ All relationships have high confidence (≥0.70) - no LLM fallback needed")

print(f"\n⏱️  Time: {elapsed*1000:.1f}ms ({elapsed*1000/len(test_relationships):.1f}ms per relationship)")

# ===== TEST 3: Hybrid Mode (if Ollama available) =====
if ollama_running and model_available:
    print("\n" + "=" * 70)
    print("TEST 3: Hybrid Categorization (Keyword + LLM Fallback)")
    print("=" * 70)
    print("⏱️  Note: LLM calls may take 5-10 seconds total...\n")

    start_time = time.time()
    hybrid_results = []
    llm_call_count = 0

    for relationship in test_relationships:
        content = relationship.get('content', '')
        rel_types = extract_relationship_types(content)
        display_name = rel_types if rel_types else content[:50]

        keyword_cat, keyword_conf = categorize_relationship_with_confidence(content)

        if keyword_conf >= 0.70:
            category, confidence = keyword_cat, keyword_conf
            used_llm = False
        else:
            category, confidence = categorize_relationship_hybrid(content, confidence_threshold=0.70)
            used_llm = (confidence == 0.90)
            if used_llm:
                llm_call_count += 1

        hybrid_results.append((display_name, category, confidence, used_llm))

    elapsed = time.time() - start_time
    display_results(hybrid_results, "Results (Hybrid mode - keyword + LLM):", show_confidence=True, show_llm=True)

    print(f"\n🤖 LLM calls: {llm_call_count}/{len(test_relationships)} ({100*llm_call_count/len(test_relationships):.1f}%)")
    print(f"⏱️  Time: {elapsed*1000:.1f}ms ({elapsed*1000/len(test_relationships):.1f}ms per relationship)")

    # ===== COMPARISON SUMMARY =====
    print("\n" + "=" * 70)
    print("📊 COMPARISON SUMMARY (Keyword vs Hybrid)")
    print("=" * 70)

    changes = 0
    for i in range(len(test_relationships)):
        if keyword_results[i][1] != hybrid_results[i][1]:
            changes += 1

    print(f"Relationships recategorized by LLM: {changes}/{len(test_relationships)} ({100*changes/len(test_relationships):.1f}%)")

    if changes > 0:
        print("\nRecategorization details:")
        for i in range(len(test_relationships)):
            kw_cat = keyword_results[i][1]
            hyb_cat = hybrid_results[i][1]
            if kw_cat != hyb_cat:
                name = keyword_results[i][0][:50]
                print(f"   - {name:50s}: {kw_cat:20s} → {hyb_cat}")

    print(f"\n✅ Hybrid categorization complete!")

else:
    print("\n" + "=" * 70)
    print("⚠️  TEST 3 SKIPPED: Ollama not available")
    print("=" * 70)
    print("To enable hybrid mode:")
    print("   1. Install Ollama: https://ollama.com")
    print(f"   2. Pull model: ollama pull {OLLAMA_MODEL_OVERRIDE}")
    print("   3. Start service: brew services start ollama (macOS)")
    print("   4. Re-run this cell")

# ===== TEST 4: Pure LLM Mode (NEW) =====
if ollama_running and model_available:
    print("\n" + "=" * 70)
    print("TEST 4: Pure LLM Categorization (All relationships)")
    print("=" * 70)
    print("⏱️  Note: This will call LLM for ALL relationships (may take 30-60 seconds)...\n")

    start_time = time.time()
    llm_results = []

    for relationship in test_relationships:
        content = relationship.get('content', '')
        rel_types = extract_relationship_types(content)
        display_name = rel_types if rel_types else content[:50]
        category, confidence = categorize_relationship_llm_only(content)
        llm_results.append((display_name, category, confidence, True))

    elapsed = time.time() - start_time
    display_results(llm_results, "Results (Pure LLM mode):", show_confidence=True, show_llm=True)

    print(f"\n🤖 LLM calls: {len(test_relationships)}/{len(test_relationships)} (100%)")
    print(f"⏱️  Time: {elapsed*1000:.1f}ms ({elapsed*1000/len(test_relationships):.1f}ms per relationship)")

    # Compare keyword vs pure LLM
    llm_changes = sum(1 for i in range(len(test_relationships)) if keyword_results[i][1] != llm_results[i][1])
    
    print("\n" + "=" * 70)
    print("📊 COMPARISON SUMMARY (Keyword vs Pure LLM)")
    print("=" * 70)
    print(f"Relationships recategorized by pure LLM: {llm_changes}/{len(test_relationships)} ({100*llm_changes/len(test_relationships):.1f}%)")
    
    if llm_changes > 0:
        print("\nRecategorization details:")
        for i in range(len(test_relationships)):
            kw_cat = keyword_results[i][1]
            llm_cat = llm_results[i][1]
            if kw_cat != llm_cat:
                name = keyword_results[i][0][:50]
                print(f"   - {name:50s}: {kw_cat:20s} → {llm_cat}")

    print(f"\n✅ Pure LLM categorization complete!")

else:
    print("\n" + "=" * 70)
    print("⚠️  TEST 4 SKIPPED: Ollama not available")
    print("=" * 70)
    print("To enable pure LLM mode:")
    print("   1. Install Ollama: https://ollama.com")
    print(f"   2. Pull model: ollama pull {OLLAMA_MODEL_OVERRIDE}")
    print("   3. Start service: brew services start ollama (macOS)")
    print("   4. Re-run this cell")

print("\n" + "=" * 70)
print("🎯 TESTING COMPLETE - 4 Categorization Modes Evaluated")
print("=" * 70)

🧪 Relationship Categorization Test (4 Modes)
✅ Categorization functions imported successfully
🔧 Ollama model configured: qwen2.5:3b

✅ Ollama service running with qwen2.5:3b model
✅ Loaded 337 relationships from knowledge graph
   Sampling mode: Reproducible (seed=42, same relationships each run)
   Testing with 12 relationships
   💡 To change: Set RANDOM_SEED = None (line 17) or OLLAMA_MODEL_OVERRIDE (line 18)


TEST 1: Keyword-Only Categorization (Baseline)

Results (Keyword Matching):
----------------------------------------------------------------------
 1. analyst recommendation,investment opport...   → Media/Analysis
 2. future prospects,growth potential             → Other
 3. financial strategy,investment solicitati...   → Financial
 4. geographic impact,service area reduction      → Impact/Correlation
 5. article source,financial news,informatio...   → Financial
 6. financial results,market performance          → Financial
 7. partnership,provides product                  → Pr

In [ ]:
# ## Clear graph storage (COMMENTED BY DEFAULT FOR SAFETY)
# ## Uncomment lines below to clear existing graph:

# ##########################################################
# #                      Clear Graph                       #
# ##########################################################

# # FIX: Reset storage_path to directory (Cell 12 redefined it to file)
# storage_path = Path(ice.config.working_dir)

# if storage_path.exists():
#     print("📊 PRE-DELETION CHECK")
#     check_storage(storage_path)
    
#     shutil.rmtree(storage_path) # Deletes directory + all contents.
#     storage_path.mkdir(parents=True, exist_ok=True) # This re-creates empty directory.
    
#     print("\n✅ POST-DELETION CHECK")
#     check_storage(storage_path)
#     print("\n✅ Graph cleared - will rebuild from scratch")
# else:
#     print("⚠️  Storage path doesn't exist - nothing to clear")

📊 PRE-DELETION CHECK
  ⚠️  vdb_entities.json: not found
  ⚠️  vdb_relationships.json: not found
  ⚠️  vdb_chunks.json: not found
  ⚠️  graph_chunk_entity_relation.graphml: not found
  💾 Total: 0.00 MB

✅ POST-DELETION CHECK
  ⚠️  vdb_entities.json: not found
  ⚠️  vdb_relationships.json: not found
  ⚠️  vdb_chunks.json: not found
  ⚠️  graph_chunk_entity_relation.graphml: not found
  💾 Total: 0.00 MB

✅ Graph cleared - will rebuild from scratch


In [10]:
# Portfolio configuration
import pandas as pd

# portfolio_df = pd.read_csv('portfolio_holdings.csv')
portfolio_df = pd.read_csv('portfolio_holdings_folder/portfolio_holdings_diversified_10.csv')

# Basic validation
if portfolio_df.empty:
    raise ValueError("Portfolio CSV is empty")
if 'ticker' not in portfolio_df.columns:
    raise ValueError("CSV must have 'ticker' column")

holdings = portfolio_df['ticker'].tolist()

print(f"🎯 Portfolio Configuration")
print(f"━" * 40)
print(f"Holdings: {', '.join(holdings)} ({len(holdings)} stocks)")
print(f"Sector: {portfolio_df['sector'].iloc[0] if len(portfolio_df) > 0 else 'N/A'}")
print(f"Data Range: 2 years historical (editable in Cell 21)")
print(f"📄 Source: portfolio_holdings.csv")

🎯 Portfolio Configuration
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
Holdings: AAPL, MSFT, NVDA, JNJ, UNH, LLY, JPM, V, XOM, CVX (10 stocks)
Sector: Technology
Data Range: 2 years historical (editable in Cell 21)
📄 Source: portfolio_holdings.csv


## 3. Data Ingestion & Processing

In [11]:
print("\n📊 Data Source Summary")
print("=" * 50)

# Show ACTUAL metrics if available (not fake percentages)
if ice and ice.core.is_ready():
    storage_stats = ice.core.get_storage_stats()
    print(f"💾 Current Graph Size: {storage_stats['total_storage_bytes'] / (1024*1024):.2f} MB")
    
    # Show real source info if ingestion has run
    if 'ingestion_result' in locals() and 'metrics' in ingestion_result:
        metrics = ingestion_result['metrics']
        if 'data_sources_used' in metrics:
            print(f"✅ Active sources: {', '.join(metrics['data_sources_used'])}")
        print(f"📄 Total documents: {ingestion_result.get('total_documents', 0)}")
    else:
        print("ℹ️ Data source metrics available after ingestion completes")
else:
    print("⚠️ Knowledge graph not ready")


📊 Data Source Summary
💾 Current Graph Size: 0.00 MB
ℹ️ Data source metrics available after ingestion completes


## 3b. Data Source Contribution Visualization (Week 5)

In [12]:
ice.core.get_graph_stats()

{'is_ready': True,
 'storage_indicators': {'all_components_present': False,
  'chunks_file_size': 0.0,
  'entities_file_size': 0.0,
  'relationships_file_size': 0.0,
  'graph_file_size': 0.0}}

In [13]:
print("📊 ICE Data Sources Summary (Phase 1 Integration)")
print("=" * 60)
print("\nℹ️  Phase 1 focuses on architecture and data flow patterns.")
print("Actual data ingestion depends on configured API keys.\n")
print("\n1️⃣ API/MCP Sources:")
print("   - NewsAPI: Real-time financial news")
print("   - SEC EDGAR: Regulatory filings (10-K, 10-Q, 8-K)")
print("   - Alpha Vantage: Market data")
print("\n2️⃣ Email Pipeline (Phase 1 Enhanced Documents):")
print("   - Broker research with BUY/SELL signals")
print("   - Enhanced documents: [TICKER:NVDA|confidence:0.95]")
print("   - See detailed demo: imap_email_ingestion_pipeline/investment_email_extractor_simple.ipynb")
print("\n3️⃣ SEC Filings:")
print("   - Management commentary and financial statements")
print("   - Integrated via SEC EDGAR connector")
print("\n💡 All sources → Single LightRAG knowledge graph via ice_simplified.py")

📊 ICE Data Sources Summary (Phase 1 Integration)

ℹ️  Phase 1 focuses on architecture and data flow patterns.
Actual data ingestion depends on configured API keys.


1️⃣ API/MCP Sources:
   - NewsAPI: Real-time financial news
   - SEC EDGAR: Regulatory filings (10-K, 10-Q, 8-K)
   - Alpha Vantage: Market data

2️⃣ Email Pipeline (Phase 1 Enhanced Documents):
   - Broker research with BUY/SELL signals
   - Enhanced documents: [TICKER:NVDA|confidence:0.95]
   - See detailed demo: imap_email_ingestion_pipeline/investment_email_extractor_simple.ipynb

3️⃣ SEC Filings:
   - Management commentary and financial statements
   - Integrated via SEC EDGAR connector

💡 All sources → Single LightRAG knowledge graph via ice_simplified.py


## 3a. ICE Data Sources Integration (Week 5)

ICE integrates 3 heterogeneous data sources into unified knowledge graph:

**Detailed Demonstrations Available**:
- 📧 **Email Pipeline**: See `imap_email_ingestion_pipeline/investment_email_extractor_simple.ipynb` (25 cells)
  - Entity extraction (tickers, ratings, price targets)
  - BUY/SELL signal extraction with confidence scores
  - Enhanced document creation with inline metadata
  
- 📊 **Quick Summary Below**

In [14]:
# Execute data ingestion
# NOTE: This operation may take several minutes. If it hangs, restart kernel.
print(f"\n📥 Fetching Portfolio Data")
print(f"━" * 50)

if not (ice and ice.is_ready()):
    raise RuntimeError("ICE system not ready for data ingestion")

# Fetch historical data (1 year for faster processing - adjust years parameter as needed)
print(f"🔄 Fetching data for {len(holdings)} holdings...")
ingestion_result = ice.ingest_historical_data(holdings, years=1) # parameter

# Display results
print(f"\n📊 Ingestion Results:")
print(f"  Status: {ingestion_result['status']}")
print(f"  Holdings: {len(ingestion_result['holdings_processed'])}/{len(holdings)}")
print(f"  Documents: {ingestion_result['total_documents']}")

# Show successful holdings
if ingestion_result['holdings_processed']:
    print(f"  ✅ Successful: {', '.join(ingestion_result['holdings_processed'])}")

# Show metrics
if 'metrics' in ingestion_result:
    print(f"\n⏱️  Processing Time: {ingestion_result['metrics']['processing_time']:.2f}s")
    if 'data_sources_used' in ingestion_result['metrics']:
        print(f"  Data Sources: {', '.join(ingestion_result['metrics']['data_sources_used'])}")
    
    # Display investment signals from Phase 2.6.1 EntityExtractor
    if 'investment_signals' in ingestion_result['metrics']:
        signals = ingestion_result['metrics']['investment_signals']
        print(f"\n📧 Investment Signals Captured:")
        print(f"  Broker emails: {signals['email_count']}")
        print(f"  Tickers covered: {signals['tickers_covered']}")
        print(f"  BUY ratings: {signals['buy_ratings']}")
        print(f"  SELL ratings: {signals['sell_ratings']}")
        print(f"  Avg confidence: {signals['avg_confidence']:.2f}")

# Show failures if any
if ingestion_result.get('failed_holdings'):
    print(f"\n❌ Failed Holdings:")
    for failure in ingestion_result['failed_holdings']:
        print(f"  {failure['symbol']}: {failure['error']}")

INFO:updated_architectures.implementation.ice_simplified:Starting historical data ingestion for 10 holdings (1 years)
INFO:updated_architectures.implementation.ice_simplified:Fetching 1 years of historical data for AAPL



📥 Fetching Portfolio Data
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
🔄 Fetching data for 10 holdings...


INFO:updated_architectures.implementation.ice_simplified:Fetched company overview for AAPL from Alpha Vantage
INFO:updated_architectures.implementation.ice_simplified:Fetched 5 news articles for AAPL from NewsAPI
INFO:updated_architectures.implementation.ice_simplified:Fetched 6 total documents for AAPL
INFO:src.ice_lightrag.model_provider:✅ Using OpenAI provider (gpt-4o-mini)
INFO: [_] Created new empty graph fiel: ice_lightrag/storage/graph_chunk_entity_relation.graphml
INFO:nano-vectordb:Init {'embedding_dim': 1536, 'metric': 'cosine', 'storage_file': 'ice_lightrag/storage/vdb_entities.json'} 0 data
INFO:nano-vectordb:Init {'embedding_dim': 1536, 'metric': 'cosine', 'storage_file': 'ice_lightrag/storage/vdb_relationships.json'} 0 data
INFO:nano-vectordb:Init {'embedding_dim': 1536, 'metric': 'cosine', 'storage_file': 'ice_lightrag/storage/vdb_chunks.json'} 0 data
INFO: [_] Process 40630 KV load full_docs with 0 records
INFO: [_] Process 40630 KV load text_chunks with 0 records
INFO:


📊 Ingestion Results:
  Status: success
  Holdings: 10/10
  Documents: 60
  ✅ Successful: AAPL, MSFT, NVDA, JNJ, UNH, LLY, JPM, V, XOM, CVX

⏱️  Processing Time: 1321.99s
  Data Sources: alpha_vantage, newsapi, polygon, finnhub


## 4. Knowledge Graph Building Pipeline

In [15]:
# Knowledge Graph Building - Already completed during ingestion
print(f"\n🧠 Knowledge Graph Building")
print(f"━" * 60)

if not (ice and ice.core.is_ready()):
    raise RuntimeError("LightRAG not ready")

print(f"ℹ️  NOTE: Knowledge graph building happens automatically during data ingestion")
print(f"   The ingestion method (ingest_historical_data) already added documents")
print(f"   to the graph via LightRAG. This cell validates that building succeeded.\n")

# Validate that building succeeded by checking storage
storage_stats = ice.core.get_storage_stats()

if storage_stats['total_storage_bytes'] > 0:
    print(f"✅ KNOWLEDGE GRAPH BUILT SUCCESSFULLY")
    print(f"━" * 40)
    print(f"   📄 Documents processed: {ingestion_result.get('total_documents', 0)}")
    print(f"   💾 Storage size: {storage_stats['total_storage_bytes'] / (1024*1024):.2f} MB")
    
    components_ready = sum(1 for c in storage_stats['components'].values() if c['exists'])
    print(f"   🔗 Components ready: {components_ready}/4")
    
    # Create success result for metrics tracking
    building_result = {
        'status': 'success',
        'total_documents': ingestion_result.get('total_documents', 0),
        'metrics': {
            'building_time': ingestion_result.get('metrics', {}).get('processing_time', 0.0),
            'graph_initialized': True
        }
    }
    
    print(f"\n🎯 Graph Building Process:")
    print(f"   1️⃣ Text Chunking: 1200 tokens (optimal for financial documents)")
    print(f"   2️⃣ Entity Extraction: Companies, metrics, risks, regulations")
    print(f"   3️⃣ Relationship Discovery: Dependencies, impacts, correlations")
    print(f"   4️⃣ Graph Construction: LightRAG optimized structure")
    print(f"   5️⃣ Storage: chunks_vdb, entities_vdb, relationships_vdb, graph")
    
    print(f"\n🚀 System ready for intelligent queries!")
    
else:
    print(f"⚠️ NO GRAPH DATA DETECTED")
    print(f"   Storage size: 0 MB")
    print(f"   Check ingestion results above for errors")
    print(f"   Possible causes:")
    print(f"   - No API keys configured")
    print(f"   - All holdings failed to fetch data")
    print(f"   - Network connectivity issues")
    
    building_result = {
        'status': 'error',
        'message': 'No graph data - check ingestion results'
    }


🧠 Knowledge Graph Building
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
ℹ️  NOTE: Knowledge graph building happens automatically during data ingestion
   The ingestion method (ingest_historical_data) already added documents
   to the graph via LightRAG. This cell validates that building succeeded.

✅ KNOWLEDGE GRAPH BUILT SUCCESSFULLY
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
   📄 Documents processed: 60
   💾 Storage size: 11.29 MB
   🔗 Components ready: 4/4

🎯 Graph Building Process:
   1️⃣ Text Chunking: 1200 tokens (optimal for financial documents)
   2️⃣ Entity Extraction: Companies, metrics, risks, regulations
   3️⃣ Relationship Discovery: Dependencies, impacts, correlations
   4️⃣ Graph Construction: LightRAG optimized structure
   5️⃣ Storage: chunks_vdb, entities_vdb, relationships_vdb, graph

🚀 System ready for intelligent queries!


## 5. Storage Architecture Validation & Monitoring

In [16]:
# Comprehensive storage validation and metrics
print(f"\n🔍 Storage Architecture Validation")
print(f"━" * 40)

if not (ice and ice.core.is_ready()):
    raise RuntimeError("Cannot validate storage without initialized system")

# Get detailed storage statistics
storage_stats = ice.core.get_storage_stats()
graph_stats = ice.core.get_graph_stats()

print(f"📦 LightRAG Storage Components Status:")
for component_name, component_info in storage_stats['components'].items():
    status_icon = "✅" if component_info['exists'] else "⚠️"
    size_mb = component_info['size_bytes'] / (1024 * 1024) if component_info['size_bytes'] > 0 else 0
    
    print(f"  {status_icon} {component_name}:")
    print(f"    File: {component_info['file']}")
    print(f"    Purpose: {component_info['description']}")
    print(f"    Size: {size_mb:.2f} MB" if size_mb > 0 else "    Size: Not created yet")

print(f"\n📊 Storage Summary:")
print(f"  Working Directory: {storage_stats['working_dir']}")
print(f"  Total Storage: {storage_stats['total_storage_bytes'] / (1024 * 1024):.2f} MB")
print(f"  System Initialized: {storage_stats['is_initialized']}")

print(f"\n🕸️ Knowledge Graph Status:")
print(f"  Graph Ready: {graph_stats['is_ready']}")
if graph_stats.get('storage_indicators'):
    indicators = graph_stats['storage_indicators']
    print(f"  All Components Present: {indicators['all_components_present']}")
    print(f"  Chunks Storage: {indicators['chunks_file_size']:.2f} MB")
    print(f"  Entity Storage: {indicators['entities_file_size']:.2f} MB")
    print(f"  Relationship Storage: {indicators['relationships_file_size']:.2f} MB")
    print(f"  Graph Structure: {indicators['graph_file_size']:.2f} MB")

# Validation checks
print(f"\n✅ Validation Checks:")
validation_score = 0
max_score = 4

# Check 1: System ready
if storage_stats['is_initialized']:
    print(f"  ✅ System initialization: PASSED")
    validation_score += 1
else:
    print(f"  ❌ System initialization: FAILED")

# Check 2: Storage exists
if storage_stats['storage_exists']:
    print(f"  ✅ Storage directory: PASSED")
    validation_score += 1
else:
    print(f"  ❌ Storage directory: FAILED")

# Check 3: Components created
components_exist = sum(1 for c in storage_stats['components'].values() if c['exists'])
if components_exist > 0:
    print(f"  ✅ Storage components: PASSED ({components_exist}/4 created)")
    validation_score += 1
else:
    print(f"  ❌ Storage components: FAILED (no components created)")

# Check 4: Has storage content
if storage_stats['total_storage_bytes'] > 0:
    print(f"  ✅ Storage content: PASSED")
    validation_score += 1
else:
    print(f"  ❌ Storage content: FAILED (no data stored)")

print(f"\n📊 Validation Score: {validation_score}/{max_score} ({(validation_score/max_score)*100:.0f}%)")

if validation_score == max_score:
    print(f"🎉 All validations passed! Knowledge graph is ready for queries.")
elif validation_score >= max_score * 0.75:
    print(f"✅ Most validations passed. System is functional.")
else:
    print(f"⚠️ Some validations failed. Check configuration and retry building.")


🔍 Storage Architecture Validation
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
📦 LightRAG Storage Components Status:
  ✅ chunks_vdb:
    File: vdb_chunks.json
    Purpose: Vector database for document chunks
    Size: 0.72 MB
  ✅ entities_vdb:
    File: vdb_entities.json
    Purpose: Vector database for extracted entities
    Size: 4.40 MB
  ✅ relationships_vdb:
    File: vdb_relationships.json
    Purpose: Vector database for entity relationships
    Size: 4.01 MB
  ✅ graph:
    File: graph_chunk_entity_relation.graphml
    Purpose: NetworkX graph structure
    Size: 0.32 MB

📊 Storage Summary:
  Working Directory: ice_lightrag/storage
  Total Storage: 11.29 MB
  System Initialized: True

🕸️ Knowledge Graph Status:
  Graph Ready: True
  All Components Present: True
  Chunks Storage: 0.72 MB
  Entity Storage: 4.40 MB
  Relationship Storage: 4.01 MB
  Graph Structure: 0.32 MB

✅ Validation Checks:
  ✅ System initialization: PASSED
  ✅ Storage directory: PASSED
  ✅ Storage components: PASSE

## 6. Building Metrics & Performance Analysis

In [17]:
# Comprehensive building session metrics
print(f"\n📊 Building Session Metrics & Performance")
print(f"━" * 50)

session_metrics = {
    'holdings_count': len(holdings),
    'total_processing_time': 0.0,
    'documents_processed': 0,
    'building_successful': False
}

# Collect metrics from ingestion and building
if 'ingestion_result' in locals() and ingestion_result:
    if 'metrics' in ingestion_result:
        session_metrics['ingestion_time'] = ingestion_result['metrics'].get('processing_time', 0.0)
    session_metrics['documents_processed'] = ingestion_result.get('total_documents', 0)

if 'building_result' in locals() and building_result:
    if building_result.get('status') == 'success':
        session_metrics['building_successful'] = True
    if 'metrics' in building_result:
        building_time = building_result['metrics'].get('building_time', building_result['metrics'].get('update_time', 0.0))
        session_metrics['building_time'] = building_time

# Calculate total time
if 'pipeline_stats' in locals():
    session_metrics['total_processing_time'] = pipeline_stats.get('processing_time', 0.0)

print(f"🎯 Session Overview:")
print(f"  Holdings Processed: {session_metrics['holdings_count']}")
print(f"  Documents Processed: {session_metrics['documents_processed']}")
print(f"  Building Successful: {session_metrics['building_successful']}")

if session_metrics.get('ingestion_time', 0) > 0:
    print(f"\n⏱️ Performance Metrics:")
    print(f"  Data Ingestion Time: {session_metrics['ingestion_time']:.2f}s")
    if session_metrics.get('building_time', 0) > 0:
        print(f"  Graph Building Time: {session_metrics['building_time']:.2f}s")
        print(f"  Total Processing Time: {session_metrics['ingestion_time'] + session_metrics['building_time']:.2f}s")
    
    print(f"\n📈 Efficiency Analysis:")
    if session_metrics['documents_processed'] > 0:
        docs_per_second = session_metrics['documents_processed'] / session_metrics['ingestion_time']
        print(f"  Processing Rate: {docs_per_second:.2f} documents/second")
    
    holdings_per_second = session_metrics['holdings_count'] / session_metrics['ingestion_time']
    print(f"  Holdings Rate: {holdings_per_second:.2f} holdings/second")

# Architecture efficiency comparison
print(f"\n🏗️ Architecture Efficiency:")
print(f"  ICE Simplified: 2,508 lines of code")
print(f"  Code Reduction: 83% (vs 15,000 line original)")
print(f"  Files Count: 5 core modules")
print(f"  Dependencies: Direct LightRAG wrapper")
print(f"  Token Efficiency: 4,000x better than GraphRAG")

# Success summary
print(f"\n✅ Building Session Summary:")
if session_metrics['building_successful']:
    print(f"  🎉 Knowledge graph building completed successfully")
    print(f"  📊 {session_metrics['documents_processed']} documents processed")
    print(f"  🚀 System ready for intelligent investment queries")
    print(f"  💡 Proceed to ice_query_workflow.ipynb for analysis")
else:
    print(f"  ⚠️ Building completed with warnings or in demo mode")
    print(f"  📋 Review configuration and API settings")
    print(f"  🔧 Consider running with fresh data if issues persist")

print(f"\n🔗 Next Steps:")
print(f"  1. Review building metrics and validate storage")
print(f"  2. Run ice_query_workflow.ipynb for portfolio analysis")
print(f"  3. Test different LightRAG query modes")
print(f"  4. Monitor system performance and optimize as needed")


📊 Building Session Metrics & Performance
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
🎯 Session Overview:
  Holdings Processed: 10
  Documents Processed: 60
  Building Successful: True

⏱️ Performance Metrics:
  Data Ingestion Time: 1321.99s
  Graph Building Time: 1321.99s
  Total Processing Time: 2643.97s

📈 Efficiency Analysis:
  Processing Rate: 0.05 documents/second
  Holdings Rate: 0.01 holdings/second

🏗️ Architecture Efficiency:
  ICE Simplified: 2,508 lines of code
  Code Reduction: 83% (vs 15,000 line original)
  Files Count: 5 core modules
  Dependencies: Direct LightRAG wrapper
  Token Efficiency: 4,000x better than GraphRAG

✅ Building Session Summary:
  🎉 Knowledge graph building completed successfully
  📊 60 documents processed
  🚀 System ready for intelligent investment queries
  💡 Proceed to ice_query_workflow.ipynb for analysis

🔗 Next Steps:
  1. Review building metrics and validate storage
  2. Run ice_query_workflow.ipynb for portfolio analysis
  3. Test diff

## 📋 Building Workflow Complete

**Summary**: This notebook demonstrated the complete ICE building workflow from data ingestion through knowledge graph construction.

### Key Achievements
✅ **System Initialization**: ICE simplified architecture deployed  
✅ **Data Ingestion**: Portfolio data fetched and processed  
✅ **Graph Building**: LightRAG knowledge graph constructed  
✅ **Storage Validation**: All components verified and metrics tracked  

### Architecture Benefits
- **83% Code Reduction**: 2,508 lines vs 15,000 original
- **4,000x Token Efficiency**: vs GraphRAG baseline
- **Mode Flexibility**: Initial build or incremental updates
- **Complete Metrics**: Processing time, success rates, storage stats

### Next Steps
1. **Launch Query Workflow**: Open `ice_query_workflow.ipynb`
2. **Test Investment Intelligence**: Run portfolio analysis queries
3. **Explore Query Modes**: Test all 5 LightRAG modes
4. **Monitor Performance**: Track query response times and accuracy

---
**Ready for Investment Intelligence Queries** 🚀